# Cutting Stock: Column Generation vs Branch-and-Price

This notebook compares two approaches for solving the Cutting Stock Problem:

1. **Column Generation (CG)** - Solves LP relaxation, then rounds to integer
2. **Branch-and-Price (B&P)** - Full branch-and-bound tree with column generation at each node

## Problem Description

Given:
- A stock roll of width $W$
- $n$ item types with widths $w_i$ and demands $d_i$

Find the minimum number of stock rolls to cut to satisfy all demands.

## Column Generation Approach

**Master Problem** (Set Covering):
```
min  sum_p x_p                   (minimize rolls)
s.t. sum_p a_ip * x_p >= d_i     (meet demand)
     x_p >= 0
```

**Pricing Problem** (Bounded Knapsack):
```
max  sum_i pi_i * y_i            (maximize dual value)
s.t. sum_i s_i * y_i <= W        (fit in roll)
     y_i integer
```

In [ ]:
# OpenCG should be installed as a package (pip install -e path/to/opencg or pip install opencg)
# OpenBP should also be installed (pip install -e path/to/openbp)

# If running from the openbp repo without installation, uncomment:
# import sys
# sys.path.insert(0, "../..")  # Add openbp root

In [ ]:
# Import OpenCG for column generation
from opencg.applications.cutting_stock import (
    CuttingStockInstance,
    solve_cutting_stock,
)

# Import OpenBP for branch-and-price
from openbp.applications.cutting_stock import (
    solve_cutting_stock_bp,
    CuttingStockBPConfig,
)

import time
import math

## Example 1: Classic Gilmore-Gomory Instance

This is a classic cutting stock instance from Gilmore & Gomory (1961).

In [ ]:
# Classic Gilmore-Gomory instance
instance = CuttingStockInstance(
    roll_width=100,
    item_sizes=[45, 36, 31, 14],
    item_demands=[97, 610, 395, 211],
    name="Gilmore-Gomory"
)

print(f"Instance: {instance.name}")
print(f"Roll width: {instance.roll_width}")
print(f"Number of item types: {instance.num_items}")
print(f"Total demand: {instance.total_demand}")
print()
print("Items:")
for i in range(instance.num_items):
    max_fit = int(instance.roll_width // instance.item_sizes[i])
    print(f"  Type {i}: size={instance.item_sizes[i]}, demand={instance.item_demands[i]}, max_per_roll={max_fit}")

In [ ]:
# Solve with column generation
start = time.time()
solution = solve_cutting_stock(instance, max_iterations=50, verbose=True)
solve_time = time.time() - start

print(f"\n" + "="*60)
print("COLUMN GENERATION RESULTS")
print("="*60)
print(f"LP relaxation: {solution.lp_objective:.4f} rolls")
print(f"IP solution:   {solution.num_rolls_ip} rolls")
print(f"L2 lower bound: {solution.lower_bound}")
print(f"Integrality gap: {100*(solution.num_rolls_ip - solution.lp_objective)/solution.lp_objective:.2f}%")
print(f"Solve time: {solve_time:.3f}s")
print(f"CG iterations: {solution.iterations}")
print(f"Patterns generated: {len(solution.patterns)}")

## Analyzing the Integrality Gap

The **integrality gap** is the difference between:
- LP relaxation (fractional patterns allowed)
- IP solution (integer patterns only)

For cutting stock, this gap is often very small (< 1%).

In [ ]:
# Calculate bounds
lp_bound = solution.lp_objective
l2_bound = solution.lower_bound
ip_value = solution.num_rolls_ip

print("Bounds Analysis:")
print("-"*40)
print(f"L2 Lower Bound (sum(sizes)/W): {l2_bound}")
print(f"LP Lower Bound:                {lp_bound:.4f}")
print(f"IP Solution:                   {ip_value}")
print()
print(f"LP-IP Gap: {ip_value - lp_bound:.4f} rolls ({100*(ip_value - lp_bound)/lp_bound:.2f}%)")
print()

if ip_value == math.ceil(lp_bound):
    print("The IP solution equals ceil(LP), which is OPTIMAL!")
    print("No branch-and-price needed for this instance.")
else:
    print("There's a gap - branch-and-price might find a better solution.")

## The Cutting Stock Integrality Property

A key theorem by Marcotte (1986):

> For cutting stock with uniform roll width, the optimal IP value is at most ceil(LP) + 1.

This means the gap is at most 2 rolls, regardless of instance size!

### When is Branch-and-Price Useful?

1. **Gap = 0**: ceil(LP) = IP optimal. B&P proves optimality but finds same solution.
2. **Gap = 1**: Either ceil(LP) is optimal, or we need 1 more roll. B&P can prove which.
3. **Theoretical interest**: B&P provides a proven optimal solution.

In [ ]:
# Show patterns in solution
print("\nPatterns in Solution:")
print("="*60)
for i, (pattern, count) in enumerate(solution.patterns):
    waste = instance.roll_width - sum(
        instance.item_sizes[j] * pattern[j] 
        for j in range(len(pattern))
    )
    print(f"Pattern {i+1}: {pattern} x {count} (waste: {waste})")

## Example 2: Instance with Potential Gap

Let's create an instance where the LP relaxation might be farther from integer.

In [ ]:
# Instance designed to have potential gap
instance2 = CuttingStockInstance(
    roll_width=100,
    item_sizes=[51, 49, 26, 25, 24],
    item_demands=[7, 7, 10, 10, 10],
    name="Gap-Instance"
)

print(f"Instance: {instance2.name}")
print(f"Roll width: {instance2.roll_width}")
print(f"Items: {list(zip(instance2.item_sizes, instance2.item_demands))}")
print(f"Total demand: {instance2.total_demand}")

In [ ]:
# Solve
sol2 = solve_cutting_stock(instance2, max_iterations=50, verbose=False)

print("\nResults:")
print(f"  LP relaxation: {sol2.lp_objective:.4f}")
print(f"  IP solution: {sol2.num_rolls_ip}")
print(f"  Gap: {100*(sol2.num_rolls_ip - sol2.lp_objective)/sol2.lp_objective:.2f}%")

# Check if optimal
if sol2.num_rolls_ip == math.ceil(sol2.lp_objective):
    print("\nIP = ceil(LP), proven OPTIMAL!")
else:
    print(f"\nIP > ceil(LP) by {sol2.num_rolls_ip - math.ceil(sol2.lp_objective)}")
    print("Branch-and-price could potentially improve this.")

## Example 3: Multiple Instances Comparison

Let's analyze integrality gaps across multiple instances.

In [ ]:
import random
random.seed(42)

# Generate random instances
instances = []
for i in range(5):
    n_items = random.randint(4, 8)
    roll_width = 100
    sizes = [random.randint(10, 45) for _ in range(n_items)]
    demands = [random.randint(5, 50) for _ in range(n_items)]
    instances.append(CuttingStockInstance(
        roll_width=roll_width,
        item_sizes=sizes,
        item_demands=demands,
        name=f"random_{i+1}"
    ))

# Solve all
print(f"{'Instance':<15} {'LP':<10} {'ceil(LP)':<10} {'IP':<10} {'Gap %':<10} {'Optimal?'}")
print("-"*70)

for inst in instances:
    sol = solve_cutting_stock(inst, max_iterations=50, verbose=False)
    lp = sol.lp_objective
    ceil_lp = math.ceil(lp)
    ip = sol.num_rolls_ip
    gap = 100 * (ip - lp) / lp if lp > 0 else 0
    optimal = "Yes" if ip == ceil_lp else "Maybe"
    
    print(f"{inst.name:<15} {lp:<10.2f} {ceil_lp:<10} {ip:<10} {gap:<10.2f} {optimal}")

## Branch-and-Price in Action

Now let's demonstrate OpenBP's Branch-and-Price solver on a cutting stock instance. B&P explores a search tree where each node solves column generation with additional branching constraints.

In [ ]:
## Summary

| Aspect | Column Generation | Branch-and-Price |
|--------|-------------------|------------------|
| Speed | Fast | Slower |
| Solution Type | LP + IP heuristic | Proven optimal IP |
| Gap Info | Knows LP bound | Closes gap completely |
| When to Use | Quick solutions | Need certified optimum |

For cutting stock, CG with IP rounding is usually sufficient. B&P adds value when:
- You need to **prove** the solution is optimal
- The IP heuristic returns ceil(LP) + 1 and you suspect ceil(LP) is optimal
- You're benchmarking algorithm performance

### OpenBP Implementation

OpenBP's cutting stock B&P uses:
- **Variable branching** on pattern usage (x_p ≤ k or x_p ≥ k+1)
- **Best-first node selection** to find optimal solutions quickly
- **Pattern-based tracking** to enforce bounds across the tree
- **Column pool** sharing for warm-starting nodes

In [ ]:
# Now solve with Branch-and-Price
print("\n" + "="*60)
print("BRANCH-AND-PRICE SOLVER")
print("="*60 + "\n")

config = CuttingStockBPConfig(
    verbose=True,
    max_nodes=100,  # Limit nodes for demo
    max_time=30.0,  # Time limit
)

start = time.time()
bp_sol = solve_cutting_stock_bp(bp_instance, config)
bp_time = time.time() - start

print("\n" + "="*60)
print("COMPARISON: CG vs B&P")
print("="*60)
print(f"{'Metric':<25} {'CG':<15} {'B&P':<15}")
print("-"*55)
print(f"{'LP Relaxation':<25} {cg_sol.lp_objective:<15.4f} {bp_sol.lower_bound:<15.4f}")
print(f"{'IP Solution':<25} {cg_sol.num_rolls_ip:<15} {int(bp_sol.objective):<15}")
print(f"{'Status':<25} {'Heuristic':<15} {bp_sol.status.name:<15}")
print(f"{'Nodes Explored':<25} {'1':<15} {bp_sol.nodes_explored:<15}")

## How B&P Works for Cutting Stock

The Branch-and-Price algorithm:

1. **Root Node**: Solve column generation to get LP relaxation
2. **Branching**: If LP solution is fractional, branch on a pattern:
   - Left child: Pattern usage ≤ floor(value)
   - Right child: Pattern usage ≥ ceil(value)
3. **Column Generation at Each Node**: Re-solve CG with branching constraints
4. **Pruning**: Discard nodes with bounds worse than best known integer solution
5. **Termination**: When all nodes are processed or pruned

### Key Observations

- For cutting stock, the integrality gap is typically very small (< 1%)
- B&P often proves that the CG heuristic solution is optimal
- The Marcotte bound guarantees IP ≤ ceil(LP) + 1

## When Would Branch-and-Price Help?

### For Cutting Stock:
- **Rarely needed** - the integrality gap is typically very small
- **Proves optimality** - when you need certification that IP = optimal
- **Tight instances** - when ceil(LP) < IP and you want to verify

### For Other Problems:
Branch-and-Price is more valuable for:
- **VRP/Crew Pairing**: Set partitioning with larger gaps
- **Bin Packing with Conflicts**: Additional constraints
- **Multi-dimensional Packing**: More complex structure

### Key Takeaways:
1. Column generation gives excellent LP bounds quickly
2. For cutting stock, IP rounding is usually optimal or near-optimal
3. B&P adds value when you need proven optimality
4. For problems with larger gaps (VRP, crew pairing), B&P is essential

## Summary

| Aspect | Column Generation | Branch-and-Price |
|--------|-------------------|------------------|
| Speed | Fast | Slower |
| Solution Type | LP + IP heuristic | Proven optimal IP |
| Gap Info | Knows LP bound | Closes gap completely |
| When to Use | Quick solutions | Need certified optimum |

For cutting stock, CG with IP rounding is usually sufficient. B&P adds value when:
- You need to **prove** the solution is optimal
- The IP heuristic returns ceil(LP) + 1 and you suspect ceil(LP) is optimal
- You're benchmarking algorithm performance